In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import re
from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)
import dateutil.parser as dparser
past_last_date = False

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

year = datetime.today().year

driver = webdriver.Firefox()
driver.get('https://investor.visa.com/news/default.aspx')

data = []

ticker = 'v'
sector = get_sector(ticker)
index_ticker = '^GSPC'
 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

year = datetime.today().year
columns = ['date', 'press title', 'urls', '1d change', 'index_price','norm_price']

try:
    driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/a[1]').click()
except:
    print('Accepted cookies')

while True:

    select = Select(driver.find_element(By.XPATH, "/html/body/div[1]/div/div/form/div[2]/div/div[4]/div/div[2]/span/span[2]/div/div/div/div/div/div[1]/div/select"))
    
    time.sleep(1)
    
    select.select_by_value(str(-1))
  

    html = driver.page_source
    soup = BeautifulSoup(html) 

    articles = soup.find_all('div', attrs={'class':'ModuleItemRow ModuleItem'})
                             
    for article in articles:
        
        date = article.find('p', attrs={'class':'vs-lead'}).text.lstrip().rstrip() 
        title = article.find('span', attrs={'class':'vs-text'}).text.rstrip() 
        url = article.find('a').get('href')
        url = f'https://investor.visa.com/{url}'
        
        content = get_content_header(url, 'meta', {'name':'twitter:description'})
        
        title = re.sub("\s\s+", " ", title)

        date = dparser.parse(date,fuzzy=False).date()
        pct_change = changes_from_press(stock_hist, date, 1)
  
        if date < datetime(2014, 1, 1).date():            
            past_last_date = True
            break  
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    if past_last_date == True: break
    

driver.close()

save_data(data, ticker, sector)

In [21]:
data

[]